<a href="https://colab.research.google.com/github/richiedinc/Python/blob/master/MAISON_DNC_Stock_Prediction_Program(%5BDATA_YAHOO%5D).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# This program predicts stock prices by using machine learning models

#Install the dependencies
import  fix_yahoo_finance as fyf
from pandas_datareader import data as pdr
import numpy as np 
import datetime as dt
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR
from sklearn.model_selection import train_test_split

In [107]:
#Get the stock data
fyf.pdr_override()
df = pdr.get_data_yahoo("AMD", start ='2000-1-01', end='2019-11-7')
df.head
# Take a look at the data
print(df.head())

[*********************100%***********************]  1 of 1 downloaded
                Open      High       Low   Close  Adj Close    Volume
Date                                                                 
2000-01-03  14.96875  15.59375  14.68750  15.500     15.500   7843200
2000-01-04  15.12500  15.50000  14.59375  14.625     14.625   6290200
2000-01-05  14.53125  15.06250  14.00000  15.000     15.000   8204600
2000-01-06  15.50000  16.00000  15.25000  16.000     16.000  11489400
2000-01-07  15.40625  16.40625  15.37500  16.250     16.250   8543400


In [108]:
# Get the Adjusted Close Price
df = df[['Close']]
#Take a look at the new data
print(df.head())

             Close
Date              
2000-01-03  15.500
2000-01-04  14.625
2000-01-05  15.000
2000-01-06  16.000
2000-01-07  16.250


In [109]:
# A variable for predicting 'n' days out into the future
forecast_out = 1 #'n=30' days
#Create another column (the target or dependent variable) shifted 'n' units up
df['Prediction'] = df[['Close']].shift(-forecast_out)
#print the new data set
print(df.tail())

                Close  Prediction
Date                             
2019-10-31  33.930000   34.889999
2019-11-01  34.889999   36.290001
2019-11-04  36.290001   36.150002
2019-11-05  36.150002   35.930000
2019-11-06  35.930000         NaN


In [110]:
### Create the independent data set (X)  #######
# Convert the dataframe to a numpy array
X = np.array(df.drop(['Prediction'],1))

#Remove the last 'n' rows
X = X[:-forecast_out]
print(X)

[[15.5     ]
 [14.625   ]
 [15.      ]
 ...
 [34.889999]
 [36.290001]
 [36.150002]]


In [111]:
### Create the dependent data set (y)  #####
# Convert the dataframe to a numpy array (All of the values including the NaN's)
y = np.array(df['Prediction'])
# Get all of the y values except the last 'n' rows
y = y[:-forecast_out]
print(y)

[14.625    15.       16.       ... 36.290001 36.150002 35.93    ]


In [0]:
# Split the data into 80% training and 20% testing
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [113]:
# Create and train the Support Vector Machine (Regressor)
svr_rbf = SVR(kernel='rbf', C=1e3, gamma=0.1)
svr_rbf.fit(x_train, y_train)

SVR(C=1000.0, cache_size=200, coef0=0.0, degree=3, epsilon=0.1, gamma=0.1,
    kernel='rbf', max_iter=-1, shrinking=True, tol=0.001, verbose=False)

In [114]:
# Testing Model: Score returns the coefficient of determination R^2 of the prediction. 
# The best possible score is 1.0
svm_confidence = svr_rbf.score(x_test, y_test)
print("svm confidence: ", svm_confidence)

svm confidence:  0.9943854799027545


In [115]:
# Create and train the Linear Regression  Model
lr = LinearRegression()
# Train the model
lr.fit(x_train, y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [116]:
# Testing Model: Score returns the coefficient of determination R^2 of the prediction. 
# The best possible score is 1.0
lr_confidence = lr.score(x_test, y_test)
print("lr confidence: ", lr_confidence)

lr confidence:  0.9960550944687995


In [117]:
# Set x_forecast equal to the last 30 rows of the original data set from Adj. Close column
x_forecast = np.array(df.drop(['Prediction'],1))[-forecast_out:]
print(x_forecast)

[[35.93]]


In [118]:
# Print linear regression model predictions for the next 'n' days
lr_prediction = lr.predict(x_forecast)
print(lr_prediction)

# Print support vector regressor model predictions for the next 'n' days
svm_prediction = svr_rbf.predict(x_forecast)
print(svm_prediction)

[35.89299478]
[35.26604707]
